In [1]:
quote do: sum(1, 2, 3)

{:sum, [], [1, 2, 3]}

In [2]:
quote do: 1 + 2

{:+, [context: Elixir, import: Kernel], [1, 2]}

In [7]:
1 + 2

3

In [8]:
:erlang.+(1,2)

3

In [10]:
if(true, do: :this, else: :that)

:this

In [11]:
quote do: %{1 => 2}

{:%{}, [], [{1, 2}]}

In [12]:
quote do: to_string(:t)

{:to_string, [context: Elixir, import: Kernel], [:t]}

In [13]:
quote do: %{1 => 2}

{:%{}, [], [{1, 2}]}

In [21]:
quote do: if value, do: "True", else: "False"

{:if, [context: Elixir, import: Kernel], [{:value, [], Elixir}, [do: "True", else: "False"]]}

In [24]:
defmodule OurMacro do
  defmacro unless(expr, do: block) do
    quote do
      if !unquote(expr), do: unquote(block)
    end
  end
end

{:module, OurMacro, <<70, 79, 82, 49, 0, 0, 4, 208, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 143, 0, 0, 0, 16, 15, 69, 108, 105, 120, 105, 114, 46, 79, 117, 114, 77, 97, 99, 114, 111, 8, 95, 95, 105, 110, 102, 111, ...>>, {:unless, 2}}

In [25]:
require OurMacro

OurMacro

In [28]:
OurMacro.unless true, do: "Hi"

nil

In [30]:
OurMacro.unless false, do: "Hi"

"Hi"

When quoting more complex expressions, we can see that the code is represented in such tuples, which are often nested inside each other in a structure resembling a tre. Many language would call such representations Abstract Syntax Tree. Elixir calls then quoted expressions.

In [31]:
inner = [3,4,5]

[3, 4, 5]

In [32]:
Macro.to_string(quote do: [1,2, unquote(inner), 6])

"[1, 2, [3, 4, 5], 6]"

In [33]:
 Macro.to_string(quote do: [1, 2, unquote_splicing(inner), 6])
"[1, 2, 3, 4, 5, 6]"

"[1, 2, 3, 4, 5, 6]"

In [34]:
 quote do: %{1 => 2}

{:%{}, [], [{1, 2}]}

In [35]:
Macro.escape %{hello: :world}

{:%{}, [], [hello: :world]}

In [36]:
 quote do: %{hello: :world}

{:%{}, [], [hello: :world]}

In [37]:
defmodule Unless do
  def fun_unless(clause, do: expression) do
    if(!clause, do: expression)
  end

  defmacro macro_unless(clause, do: expression) do
    quote do
      if(!unquote(clause), do: unquote(expression))
    end
  end
end


{:module, Unless, <<70, 79, 82, 49, 0, 0, 6, 8, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 168, 0, 0, 0, 19, 13, 69, 108, 105, 120, 105, 114, 46, 85, 110, 108, 101, 115, 115, 8, 95, 95, 105, 110, 102, 111, 95, 95, ...>>, {:macro_unless, 2}}

In [39]:
 expr = quote do: Unless.macro_unless(true, do: IO.puts "this should never be printed")

{{:., [], [{:__aliases__, [alias: false], [:Unless]}, :macro_unless]}, [], [true, [do: {{:., [], [{:__aliases__, [alias: false], [:IO]}, :puts]}, [], ["this should never be printed"]}]]}

Elixir macros have late resolution. This guarantees that a variable defined inside a quote won't conflit with a variable defined in the context where that macro is expanded.

In [41]:
defmodule Hygiene do
  defmacro no_interference do
    quote do: a = 1
  end
end

defmodule HygieneTest do
  def go do
   require Hygiene
   a = 13
   Hygiene.no_interference
   a
  end
end

{:module, HygieneTest, <<70, 79, 82, 49, 0, 0, 4, 36, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 128, 0, 0, 0, 13, 18, 69, 108, 105, 120, 105, 114, 46, 72, 121, 103, 105, 101, 110, 101, 84, 101, 115, 116, 8, 95, 95, 105, ...>>, {:go, 0}}

In [42]:
HygieneTest.go

13